# ECS7020P mini-project submission

The mini-project has two separate components:


1.   **Basic component** [6 marks]: Using the MLEnd London Sounds dataset, build a machine learning pipeline that takes as an input an audio segment and predicts whether the audio segment has been recorded indoors or outdoors.
2.   **Advanced component** [10 marks]: Formulate your own machine learning problem and build a machine learning solution using the MLEnd London Sounds dataset. 

Your submission will consist of two Jupyter notebooks, one for the basic component and another one for advanced component. Please **name each notebook**:

* ECS7020P_miniproject_basic.ipynb
* ECS7020P_miniproject_advanced.ipynb

then **zip and submit them toghether**.

Each uploaded notebook should include: 

*   **Text cells**, describing concisely each step and results.
*   **Code cells**, implementing each step.
*   **Output cells**, i.e. the output from each code cell.

and **should have the structure** indicated below. Notebooks might not be run, please make sure that the output cells are saved.

How will we evaluate your submission?

*   Conciseness in your writing (10%).
*   Correctness in your methodology (30%).
*   Correctness in your analysis and conclusions (30%).
*   Completeness (10%).
*   Originality (10%).
*   Efforts to try something new (10%).

Suggestion: Why don't you use **GitHub** to manage your project? GitHub can be used as a presentation card that showcases what you have done and gives evidence of your data science skills, knowledge and experience. 

Each notebook should be structured into the following 9 sections:


# 1 Author

**Student Name**:  Karan Kaus
**Student ID**:  



# 2 Problem formulation

Describe the machine learning problem that you want to solve and explain what's interesting about it.

# 3 Machine Learning pipeline

Describe your ML pipeline. Clearly identify its input and output, any intermediate stages (for instance, transformation -> models), and intermediate data moving from one stage to the next. It's up to you to decide which stages to include in your pipeline. 

# 4 Transformation stage

Describe any transformations, such as feature extraction. Identify input and output. Explain why you have chosen this transformation stage.

# 5 Modelling

Describe the ML model(s) that you will build. Explain why you have chosen them.

# 6 Methodology

Describe how you will train and validate your models, how model performance is assesssed (i.e. accuracy, confusion matrix, etc)

# 7 Dataset

Describe the dataset that you will use to create your models and validate them. If you need to preprocess it, do it here. Include visualisations too. You can visualise raw data samples or extracted features.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa.display
import math

import os, sys, re, pickle, glob
import urllib.request
import zipfile

import IPython.display as ipd
from tqdm import tqdm
import librosa
import scipy.stats as stats

In [2]:
sample_path = 'MLEndLS/*.wav'
files = glob.glob(sample_path)
MLENDLS_df = pd.read_csv('./MLEndLS.csv').set_index('file_id') 
MLENDLS_df

,area,spot,in_out,Participant
file_id,,,,
0001.wav,british,street,outdoor,S151
0002.wav,kensington,dinosaur,indoor,S127
0003.wav,campus,square,outdoor,S18
0004.wav,kensington,hintze,indoor,S179
0005.wav,campus,square,outdoor,S176
...,...,...,...,...
2496.wav,westend,trafalgar,outdoor,S151
2497.wav,campus,square,outdoor,S6
2498.wav,westend,national,indoor,S96


In [14]:
import matplotlib.pyplot as plt

aud_data = []
labels = []

def getPitch(x,fs,winLen=0.02):
    #winLen = 0.02 
    p = winLen*fs
    frame_length = int(2**int(p-1).bit_length())
    hop_length = frame_length//2
    f0 = librosa.yin(y=x, fmin=300, fmax=1000, sr=fs,
                frame_length=frame_length,hop_length=hop_length)
    return f0

def getXy(files,labels_file, scale_audio=False, onlySingleDigit=False):
    
    X,y,new_files =[],[],[]
    
    for file in tqdm(files):
        fileID = file.split('/')[-1]
        file_name = file.split('/')[-1]

        yi = labels_file.loc[fileID]['in_out'] == 'indoor'

        fs = None # if None, fs would be 22050
        x, fs = librosa.load(file,sr=fs)
        f0 = None

        if scale_audio: x = x/np.max(np.abs(x))
        
        try:
            f0 = getPitch(x,fs,winLen=0.02)
            pass
        except:
            print("Invalid file", file_name)
            continue
            
        aud_data.append(d_mfcc)

        power = np.sum(x**2)/len(x)
        pitch_mean = np.nanmean(f0) if np.mean(np.isnan(f0))<1 else 0
        pitch_std  = np.nanstd(f0) if np.mean(np.isnan(f0))<1 else 0
#         voiced_fr = np.mean(voiced_flag)

        xi = [ power, pitch_mean, pitch_std, labels_file.loc[fileID]['Participant'], labels_file.loc[fileID]['spot'], labels_file.loc[fileID]['area']]
        X.append(xi)
        y.append(yi)
        new_files.append(file)

    return np.array(X),np.array(y),new_files

X, y, new_files = getXy(files[0:], labels_file=MLENDLS_df, scale_audio=True, onlySingleDigit=True)

print("\nDone")
display(X.shape)

  7%|██▌                                    | 165/2500 [00:08<01:52, 20.76it/s]/tmp/ipykernel_26488/3417206462.py:29: RuntimeWarning: invalid value encountered in divide
  if scale_audio: x = x/np.max(np.abs(x))
  7%|██▌                                    | 168/2500 [00:09<01:44, 22.22it/s]

Invalid file 2206.wav


 93%|███████████████████████████████████▏  | 2316/2500 [01:50<00:06, 27.31it/s]

Invalid file 0033.wav


100%|██████████████████████████████████████| 2500/2500 [01:58<00:00, 21.18it/s]


Done


(2498, 6)

In [16]:
X_backup=X

In [17]:
display(X[:,0:3].shape)

(2498, 3)

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

numComponents = 5
pca = PCA(n_components=numComponents)
pca.fit(aud_data)

projected = pca.transform(aud_data)
projected = pd.DataFrame(projected,columns=['pc1','pc2','pc3','pc4','pc5'],index=range(1,len(aud_data)+1))
# projected['label'] = labels
display(projected)

In [ ]:
projected = projected.drop(columns=[
#     'pc3',
    'pc4',
    'pc5',
#     'pc6',
#     'pc7'
])

In [30]:
from sklearn.model_selection import GridSearchCV, train_test_split
X = X[:,0:3]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
# X_train, X_val, y_train, y_val = train_test_split(projected,labels,test_size=0.2)

print(X_train.shape, X_val.shape)

(1998, 3) (500, 3)


# 8 Results

Carry out your experiments here, explain your results.

In [26]:
from sklearn import svm

parameters = {'C':[1,2,3,4,5]}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters,cv=5)

clf.fit(X_train,y_train)

print('Hyperparameters: ', clf.best_estimator_)
print('Average accuracy: ', clf.best_score_)
print('Test dataset accuracy:', clf.score(X_val, y_val))

Hyperparameters:  SVC(C=1)
Average accuracy:  0.7242105263157895
Test dataset accuracy: 0.706


In [33]:
parameters = {'n_estimators':[50,200,300, 500], 'max_features':[0.1,1,10], 'max_depth':[7]}

rfc = RandomForestClassifier(max_depth=7)
clf = GridSearchCV(rfc, parameters,cv=5)

clf.fit(X_train,y_train)

print('Hyperparameters: ', clf.best_estimator_)
print('Average accuracy: ', clf.best_score_)
print('Test dataset accuracy:', clf.score(X_val, y_val))

Hyperparameters:  RandomForestClassifier(max_depth=7, max_features=1, n_estimators=50)
Average accuracy:  0.730733082706767
Test dataset accuracy: 0.73


In [27]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

from sklearn.model_selection import GridSearchCV, train_test_split

X_train, X_val, y_train, y_val = train_test_split(projected,labels,test_size=0.2)
print(X_train.shape, X_val.shape)

clf = RandomForestClassifier(max_depth=7)
clf.fit(X_train,y_train)

yt_p = clf.predict(X_train)
yv_p = clf.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))

NameError: name 'projected' is not defined

In [ ]:
model  = svm.SVC(C=1)
model.fit(X_train,y_train)

yt_p = model.predict(X_train)
yv_p = model.predict(X_val)

print('Training Accuracy', np.mean(yt_p==y_train))
print('Validation  Accuracy', np.mean(yv_p==y_val))
print('The support vectors are', model.support_vectors_.shape)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

clf = DecisionTreeClassifier(max_depth=None, min_samples_split=102,
     random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)
print(scores.mean())

clf = RandomForestClassifier(n_estimators=55, max_depth=None,
     min_samples_split=2, random_state=0)
scores = cross_val_score(clf, X_train, y_train, cv=5)
print(scores.mean())

# 9 Conclusions

Your conclusions, improvements, etc should go here